In [1]:
import numpy as np
import xml.etree.ElementTree as ET
from PIL import Image
import xml.dom.minidom as minidom

In [2]:
# Converts image to 64x64
def resize_and_pad(image_path, output_path, target_size=(64, 64)):
    img = Image.open(image_path)
    original_width, original_height = img.size
    aspect_ratio = original_width / original_height

    if aspect_ratio > 1:
        new_width = target_size[0]
        new_height = round(target_size[0] / aspect_ratio)
    else:
        new_width = round(target_size[1] * aspect_ratio)
        new_height = target_size[1]

    img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
    padded_img = Image.new("RGB", target_size, color="black")
    padded_img.paste(img, ((target_size[0] - new_width) // 2, (target_size[1] - new_height) // 2))
    padded_img.save(output_path)

In [8]:
from PIL import Image

def create_image(width, height, point_x, point_y, output_path, filename):
    image = Image.new('RGB', (width, height), 'black')
    pixels = image.load()
    pixels[point_x, point_y] = (255, 255, 255)  # white pixel at the specified position
    filepath = output_path + filename + '.png'
    image.save(filepath, 'PNG')
    print('Image created successfully!')

# Input values
width = 64
height = 128
point_x = 37
point_y = 55
output_path = './images/'
filename = 'test'

point_img = create_image(width, height, point_x, point_y, output_path, filename)

Image created successfully!


In [3]:
import pandas as pd

pd.options.display.max_seq_items = 2000

def convert_image_to_array(image_path):
    # Open the image and convert it to grayscale
    image = Image.open(image_path).convert('L')

    # Convert the image to a NumPy array
    img_array = np.array(image)

    # Threshold the image to create a binary array (0 or 1)
    threshold_value = 128
    binary_array = (img_array > threshold_value).astype(int)

    return binary_array


image_path = './images/binary_mask_output.png'
binary_array  = convert_image_to_array(image_path)
print(binary_array[32])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [141]:
# Open the padded image
img = Image.open('./images/binary_mask_output.png')

# Convert to grayscale
img_gray = img.convert('L')

# Threshold to get a binary image
threshold = 128
img_binary = img_gray.point(lambda p: p > threshold and 1)

# Convert to numpy array
map_array = np.asarray(img_binary)

# Create an all-zero array of the same dimensions
heatmap_array = np.zeros_like(map_array)

In [142]:
def generate_xml(start_x, start_y, goal_x, goal_y, height, width, map_array, heatmap_array):
    root = ET.Element("root")

    # Adding map dimensions and start/goal positions
    map_element = ET.SubElement(root, "map")
    ET.SubElement(map_element, "startx").text = str(start_x)
    ET.SubElement(map_element, "starty").text = str(start_y)
    ET.SubElement(map_element, "goalx").text = str(goal_x)
    ET.SubElement(map_element, "goaly").text = str(goal_y)
    ET.SubElement(map_element, "width").text = str(width)
    ET.SubElement(map_element, "height").text = str(height)

    # Adding binary 2d array
    grid = ET.SubElement(map_element, "grid")
    for row in map_array:
        row_element = ET.SubElement(grid, "row")
        row_element.text = ' '.join(map(str, row))

    # Adding heatmap dimensions
    heatmap = ET.SubElement(root, "heatmap")
    ET.SubElement(heatmap, "width").text = str(width)
    ET.SubElement(heatmap, "height").text = str(height)

    # Adding 2d heat map array
    grid = ET.SubElement(heatmap, "grid")
    for row in heatmap_array:
        row_element = ET.SubElement(grid, "row")
        row_element.text = ' '.join(map(str, row))

    # Creating XML file
    tree = ET.ElementTree(root)
    ET.indent(tree, space='\t', level=0)

    # Saving XML file 
    tree.write("./maps/heatmap_0_k.xml", encoding="utf-8", xml_declaration=True)

generate_xml(10, 10, 34, 54, 64, 64, map_array, heatmap_array)